In [75]:
using Pkg
# Ensure required packages (uncomment to install if needed)
Pkg.add([
    "MLJ", 
    "MLJBase", 
    "MLJModels", 
    "MLJEnsembles", 
    "MLJLinearModels", 
    "DecisionTree", 
    "MLJDecisionTreeInterface", 
    "NaiveBayes", 
    "EvoTrees", 
    "CategoricalArrays", 
    "Random",
    "LIBSVM",           
    "Plots",            
    "MLJModelInterface", 
    "CSV",              
    "DataFrames",      
    "MLJFlux", 
    "UrlDownload",      
    "XGBoost"    
])
include("Utils.jl")
using .Utils
using CSV
using DataFrames

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [76]:
# 1. Load data from wdbc.data file
df = CSV.read("wdbc.data", DataFrame, header=false)

new_names = [
    "ID", "Diagnosis",
    # The Mean (first 10 features)
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave_points_mean", "symmetry_mean", "fractal_dimension_mean",
    # The Standard Error (next 10 features)
    "radius_se", "texture_se", "perimeter_se", "area_se", "smoothness_se",
    "compactness_se", "concavity_se", "concave_points_se", "symmetry_se", "fractal_dimension_se",
    # The "Worst" or Largest (last 10 features)
    "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst",
    "compactness_worst", "concavity_worst", "concave_points_worst", "symmetry_worst", "fractal_dimension_worst"
]
rename!(df, new_names)

# 4. Verify the changes
# Let's look at the first 3 rows and the new header
println("--- Updated DataFrame Headers ---")
first(df, 3)

# 2. Data separation (mimicking the Python script)
y = df[:, 2]
x = df[:, 3:end]

# 3. Metadata equivalent
println("--- Dataset Summary ---")
println("Total Rows: ", nrow(df))
println("Total Columns: ", ncol(df))
println("Target variable (y) shape: ", size(y))
println("Features matrix (X) shape: ", size(x))

println("\n--- Variable Information (First 5 Features) ---")
display(describe(x))
println("\n--- Target Distribution ---")
display(combine(groupby(DataFrame(Diagnosis=y), :Diagnosis), nrow))

--- Updated DataFrame Headers ---
--- Dataset Summary ---
Total Rows: 569
Total Columns: 32
Target variable (y) shape: (569,)
Features matrix (X) shape: (569, 30)

--- Variable Information (First 5 Features) ---


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,radius_mean,14.1273,6.981,13.37,28.11,0,Float64
2,texture_mean,19.2896,9.71,18.84,39.28,0,Float64
3,perimeter_mean,91.969,43.79,86.24,188.5,0,Float64
4,area_mean,654.889,143.5,551.1,2501.0,0,Float64
5,smoothness_mean,0.0963603,0.05263,0.09587,0.1634,0,Float64
6,compactness_mean,0.104341,0.01938,0.09263,0.3454,0,Float64
7,concavity_mean,0.0887993,0.0,0.06154,0.4268,0,Float64
8,concave_points_mean,0.0489191,0.0,0.0335,0.2012,0,Float64
9,symmetry_mean,0.181162,0.106,0.1792,0.304,0,Float64



--- Target Distribution ---


Row,Diagnosis,nrow
,String1,Int64
1,M,212
2,B,357


In [ ]:
x = Matrix(x)
classes = unique(y)
y_encoded = Utils.oneHotEncoding(y, classes)

Utils.normalizeMinMax!(x)

println("--- Data Summary (after processing with Utils) ---")
println("Features (X) dimension: ", size(X))
println("Targets (y_encoded) dimension: ", size(y_encoded))
println("Mean of first feature column: ", mean(X[:, 1]))
println("Max of first feature column: ", maximum(X[:, 1]))
println("Targets (first 5 encoded): \n", first(y_encoded, 5))

LoadError: UndefVarError: `coerce!` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing ScientificTypes in the current active module Main
Hint: a global variable of this name may be made accessible by importing MLJBase in the current active module Main

In [78]:
# num_instances = size(x, 1)
# validation_ratio = 0.1
# test_ratio = 0.1

# train_idx, val_idx, test_idx = Utils.holdOut(num_instances, validation_ratio, test_ratio)

# x_train = x[train_idx, :]
# y_train = y_encoded[train_idx, :]

# x_val = x[val_idx, :]
# y_val = y_encoded[val_idx, :]

# x_test = x[test_idx, :]
# y_test = y_encoded[test_idx, :]

# println("Total instances: $num_instances")
# println("Training set size: $(size(x_train, 1))")
# println("Validation set size: $(size(x_val, 1))")
# println("Test set size: $(size(x_test, 1))")

In [79]:
estimators = [:SVM, :DecisionTree, :kNN]

svm_params = Dict() 
dt_params  = Dict(:max_depth => 5)
knn_params = Dict(:K => 5)

model_hyperparams = [svm_params, dt_params, knn_params]

ensemble_params = Dict(:voting => :hard)

k_folds = 5
cv_indices = Utils.crossvalidation(y_encoded, k_folds)

# ---------------------------------------------------------
# 4. Train and Evaluate the Ensemble
# ---------------------------------------------------------
println("Training Ensemble ($estimators) with $k_folds-fold CV...")

# Utils.trainClassEnsemble performs the training loop and returns aggregated metrics
results = Utils.trainClassEnsemble(
    estimators, 
    model_hyperparams, 
    ensemble_params, 
    (x, y_encoded), 
    cv_indices
)

# ---------------------------------------------------------
# 5. Display Results
# ---------------------------------------------------------
# The results tuple order from Utils.jl [Source 146/169] is:
# (acc, err, sens, spec, ppv, npv, f1, globalCM) - where each metric is (mean, std)

(acc_mean, acc_std)   = results[1]
(sens_mean, sens_std) = results[3]
(spec_mean, spec_std) = results[4]
(f1_mean, f1_std)     = results[7]
confusion_matrix      = results[8]

println("\n--- Ensemble Results ---")
println("Accuracy:    $(round(acc_mean, digits=4)) ± $(round(acc_std, digits=4))")
println("Sensitivity: $(round(sens_mean, digits=4)) ± $(round(sens_std, digits=4))")
println("Specificity: $(round(spec_mean, digits=4)) ± $(round(spec_std, digits=4))")
println("F1-Score:    $(round(f1_mean, digits=4)) ± $(round(f1_std, digits=4))")

println("\nGlobal Confusion Matrix (Summed over folds):")
display(confusion_matrix)

Training Ensemble ([:SVM, :DecisionTree, :kNN]) with 5-fold CV...
metric workeds
metric workeds
metric workeds
metric workeds
metric workeds


┌ Error: Error training ensemble
│   exception =
│    BoundsError: attempt to access 0×30 Matrix{Float64} at index [1, 1:30]
│    Stacktrace:
│      [1] throw_boundserror(A::Matrix{Float64}, I::Tuple{Int64, Base.Slice{Base.OneTo{Int64}}})
│        @ Base ./essentials.jl:14
│      [2] checkbounds
│        @ ./abstractarray.jl:699 [inlined]
│      [3] _getindex
│        @ ./multidimensional.jl:914 [inlined]
│      [4] getindex
│        @ ./abstractarray.jl:1312 [inlined]
│      [5] stack_function_results(row_fun::DecisionTree.var"#33#34"{DecisionTree.Node{Float64, UInt32}, Vector{UInt32}}, X::Matrix{Float64})
│        @ DecisionTree ~/.julia/packages/DecisionTree/0Dw1P/src/classification/main.jl:28
│      [6] apply_tree_proba
│        @ ~/.julia/packages/DecisionTree/0Dw1P/src/classification/main.jl:333 [inlined]
│      [7] apply_tree_proba
│        @ ~/.julia/packages/DecisionTree/0Dw1P/src/classification/main.jl:328 [inlined]
│      [8] predict
│        @ ~/.julia/packages/MLJDecisionT

Error training ensemble: BoundsError(Matrix{Float64}(undef, 0, 30), (1, Base.Slice(Base.OneTo(30))))
Error training ensemble: BoundsError(Matrix{Float64}(undef, 0, 30), (1, Base.Slice(Base.OneTo(30))))


┌ Error: Error training ensemble
│   exception =
│    Conflicting categorical pools detected. 
│    Stacktrace:
│      [1] check_pools(A::CategoricalArrays.CategoricalVector{String, UInt32, String, CategoricalArrays.CategoricalValue{String, UInt32}, Union{}}, B::CategoricalArrays.CategoricalVector{String, UInt32, String, CategoricalArrays.CategoricalValue{String, UInt32}, Union{}})
│        @ StatisticalMeasuresBase ~/.julia/packages/StatisticalMeasuresBase/GOCHI/src/tools_for_implementers.jl:139
│      [2] _call
│        @ ~/.julia/packages/StatisticalMeasuresBase/GOCHI/src/fussy_measure.jl:43 [inlined]
│      [3] call(::StatisticalMeasuresBase.FussyMeasure{StatisticalMeasuresBase.RobustMeasure{StatisticalMeasuresBase.Multimeasure{StatisticalMeasuresBase.SupportsMissingsMeasure{StatisticalMeasures.AccuracyOnScalars}, Nothing, StatisticalMeasuresBase.Mean, typeof(identity)}}, Nothing}, ::CategoricalArrays.CategoricalVector{String, UInt32, String, CategoricalArrays.CategoricalValue{Stri

Error training ensemble: ErrorException("Conflicting categorical pools detected. ")
Error training ensemble: BoundsError(Matrix{Float64}(undef, 0, 30), (1, Base.Slice(Base.OneTo(30))))
Error training ensemble: BoundsError(Matrix{Float64}(undef, 0, 30), (1, Base.Slice(Base.OneTo(30))))


...

Error training ensemble: BoundsError(Matrix{Float64}(undef, 0, 30), (1, Base.Slice(Base.OneTo(30))))
Error training ensemble: BoundsError(Matrix{Float64}(undef, 0, 30), (1, Base.Slice(Base.OneTo(30))))


Excessive output truncated after 529305 bytes.

Error training ensemble: BoundsError(Matrix{Float64}(undef, 0, 30), (1, Base.Slice(Base.OneTo(30))))


┌ Error: Error training ensemble
│   exception =
│    BoundsError: attempt to access 0×30 Matrix{Float64} at index [1, 1:30]
│    Stacktrace:
│      [1] throw_boundserror(A::Matrix{Float64}, I::Tuple{Int64, Base.Slice{Base.OneTo{Int64}}})
│        @ Base ./essentials.jl:14
│      [2] checkbounds
│        @ ./abstractarray.jl:699 [inlined]
│      [3] _getindex
│        @ ./multidimensional.jl:914 [inlined]
│      [4] getindex
│        @ ./abstractarray.jl:1312 [inlined]
│      [5] stack_function_results(row_fun::DecisionTree.var"#33#34"{DecisionTree.Node{Float64, UInt32}, Vector{UInt32}}, X::Matrix{Float64})
│        @ DecisionTree ~/.julia/packages/DecisionTree/0Dw1P/src/classification/main.jl:28
│      [6] apply_tree_proba
│        @ ~/.julia/packages/DecisionTree/0Dw1P/src/classification/main.jl:333 [inlined]
│      [7] apply_tree_proba
│        @ ~/.julia/packages/DecisionTree/0Dw1P/src/classification/main.jl:328 [inlined]
│      [8] predict
│        @ ~/.julia/packages/MLJDecisionT

LoadError: BoundsError: attempt to access Float64 at index [2]